<h3>Imports<h3>

In [2]:
import sqlite3
import csv
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *

<h3>Configs<h3>

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better


conf = SparkConf() \
.setAppName('app') \
.setMaster("local[*]")

sc = SparkContext.getOrCreate(conf=conf)

sqlContext = SQLContext(sc)

C:\Users\10010917\Anaconda3\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


<h3>Read Data From .db<h3>

In [4]:
# Create your connection.
conn = sqlite3.connect('C:/Users/10010917/Desktop/a/test/movielens-small.db')
c = conn.cursor()

In [5]:
links = c.execute('SELECT * FROM links')
links = links.fetchall()

movies = c.execute('SELECT * FROM movies')
movies = movies.fetchall()

ratings = c.execute('SELECT * FROM ratings')
ratings = ratings.fetchall()

tags = c.execute('SELECT * FROM tags')
tags = tags.fetchall()

<h3>Convert Csv<h3>

In [5]:
with open('links.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['movieId','imdbId', 'tmdbId'])
    for row in links:
        csv_out.writerow(row)
        
with open('movies.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['movieId','title', 'year', 'genres'])
    for row in movies:
        movieId = row[0]
        title = row[1].encode(errors='ignore')
        year = row[2]
        genres = row[3]
        newRow = (movieId, title, year, genres)
        csv_out.writerow(newRow)
        
with open('ratings.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['userId','movieId', 'rating', 'timestamp'])
    for row in ratings:
        csv_out.writerow(row)
        
with open('tags.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['userId','movieId', 'tag', 'timestamp'])
    for row in tags:
        csv_out.writerow(row)

<h3>Csv to DataFrame<h3>

In [6]:
linksDf   = spark.read.format("csv").load("links.csv",   header=True).cache()
moviesDf  = spark.read.format("csv").load("movies.csv",  header=True).cache()
ratingsDf = spark.read.format("csv").load("ratings.csv", header=True).cache()
tagsDf    = spark.read.format("csv").load("tags.csv",    header=True).cache()

<h2>Join Two Table<h2>

In [ ]:
queryResultJ = moviesDf.join(ratingsDf, moviesDf.movieId == ratingsDf.movieId) \
    .select(ratingsDf.userId, moviesDf.movieId, moviesDf.genres, ratingsDf.rating)

In [ ]:
queryResultTaskJ


userId,movieId,genres,rating
1,6,Action|Crime|Thri...,2.0
1,22,Crime|Drama|Horro...,3.0
1,32,Mystery|Sci-Fi|Th...,2.0
1,50,Crime|Mystery|Thr...,5.0
1,110,Action|Drama|War,4.0
1,164,Crime|Film-Noir|M...,3.0
1,198,Action|Crime|Dram...,3.0
1,260,Action|Adventure|...,5.0
1,296,Comedy|Crime|Dram...,4.0
1,303,Action|Thriller|W...,3.0


<h2>Count ratings for each movie, and list top 5 movies with the highest value<h2>

In [ ]:
queryResultB =  queryResultTaskA.groupBy('movieId').agg(\
                                                            count('rating').alias('count_ratings'),
                                                            avg('rating').alias('avg_rating'), \
                                                            ).sort(col('count_ratings').desc())

<h2>Top 5 most rated genres<h>

In [ ]:
queryResultB.show(5)

+-------+-------------+------------------+
|movieId|count_ratings|        avg_rating|
+-------+-------------+------------------+
|    593|          337| 4.235905044510385|
|    318|          328| 4.442073170731708|
|    296|          327| 4.137614678899083|
|    480|          324|3.6018518518518516|
|    356|          318| 3.908805031446541|
+-------+-------------+------------------+
only showing top 5 rows



In [ ]:
queryResultC =  queryResultTaskA.groupBy('genres').count().sort(col("count").desc())

In [ ]:
queryResultC.show(5)

+--------------+-----+
|        genres|count|
+--------------+-----+
|         Drama| 6927|
|        Comedy| 6364|
|Comedy|Romance| 3744|
| Drama|Romance| 3114|
|  Comedy|Drama| 2993|
+--------------+-----+
only showing top 5 rows

